In [14]:
import pandas as pd
import os

# Load experiment results

In [15]:
# Definitions
metrics_path = "../results/influence_of_n_samples/performance/"
timing_path = "../results/influence_of_n_samples/timing/"
output_path = "../results/influence_of_n_samples/"
metrics = ['CRPSValidation', 'Gaussian NLLValidation',
    'CoverageValidation', 'IntervalWidthValidation', 'MSETest',
    'EnergyScoreTest', 'CRPSTest', 'Gaussian NLLTest', 'CoverageTest',
    'IntervalWidthTest']
agg_groups = ["uncertainty_quantification", "n_samples"]
n_samples = [3, 5, 10, 20, 50]
methods = ["scoring-rule-dropout", "scoring-rule-reparam"]


# Load experiment results
results = pd.DataFrame()
# Loop over subfolders
folders = os.listdir(metrics_path)
for sf in folders:
    if os.path.isdir(metrics_path + sf):
        file = os.path.join(metrics_path+sf, "test.csv")
        # Read file if exits
        if os.path.exists(file):
            results_df = pd.read_csv(file, index_col=0)
            results = pd.concat([results, results_df], axis = 1)
rows = metrics.copy()
for g in agg_groups:
    rows.append(g)

results = results.loc[rows]
results.loc[metrics] = results.loc[metrics].astype("float32")
results.loc["n_samples"] = results.loc["n_samples"].astype("int32")
results = results.transpose()
# Group by uncertainty quantification method
mean = results.groupby(agg_groups).mean().astype("float32")
mean.insert(0, "Statistic", "Mean")
std = results.groupby(agg_groups).std().astype("float32")
std.insert(0, "Statistic", "Std")

results_df = pd.concat([mean.transpose(), std.transpose()], axis = 1)
results_df = results_df[results_df.columns.sort_values().unique()]

In [16]:
# Add Training time results
results_df.loc["training_time"] = 0
results_timing = pd.DataFrame()
# Loop over subfolders
folders = os.listdir(timing_path)
for sf in folders:
    if os.path.isdir(timing_path + sf):
        file = os.path.join(timing_path + sf, "test.csv")
        # Read file if exits
        if os.path.exists(file):
            results_df_timing = pd.read_csv(file, index_col=0)
            results_timing = pd.concat([results_timing, results_df_timing], axis=1)
rows = ["t_training"]
for g in agg_groups:
    rows.append(g)
results_timing = results_timing.loc[rows].transpose()
results_timing["t_training"] = results_timing["t_training"].astype("float32")
results_timing["n_samples"] = results_timing["n_samples"].astype("int32")
for method in methods:
    for n in n_samples:
        results_df.loc["training_time", (method, n)]  = results_timing[
            (results_timing["uncertainty_quantification"] == method)
            & (results_timing["n_samples"] == n)
        ]["t_training"].values[0] / 25 # Number of epochs
results_df.sort_values(by = ["uncertainty_quantification", "n_samples"], axis = 1, inplace = True)
results_df.to_csv(f"{output_path}aggregated_results.csv")

# Create latex table

In [17]:
results_df

uncertainty_quantification scoring-rule-dropout                        \
n_samples                                    3          3          5    
Statistic                                  Mean        Std       Mean   
CRPSValidation                         0.548556   0.009714    0.55685   
Gaussian NLLValidation                 3.032448   0.585035   5.293431   
CoverageValidation                     0.765348    0.01877   0.754921   
IntervalWidthValidation                3.091159   0.041356   3.110771   
MSETest                                0.879336   0.007243   0.878956   
EnergyScoreTest                        0.619538   0.004923   0.619132   
CRPSTest                               0.549622   0.009735   0.557674   
Gaussian NLLTest                       3.038941   0.587175   5.287218   
CoverageTest                           0.764044    0.01913   0.754054   
IntervalWidthTest                      3.085179   0.042239   3.106808   
training_time                         37.723999  37.723999  44.584038   

uncertainty_quantification                                              \
n_samples                          5          10         10         20   
Statistic                         Std       Mean        Std       Mean   
CRPSValidation               0.007986   0.534053   0.003994   0.555692   
Gaussian NLLValidation       1.894548   2.348024   0.223255   5.315161   
CoverageValidation           0.018761   0.800707   0.009472   0.760151   
IntervalWidthValidation      0.054213   3.130034   0.037591   3.135739   
MSETest                      0.005881   0.879897   0.002117   0.878388   
EnergyScoreTest              0.003831   0.619649   0.001533   0.618801   
CRPSTest                     0.007961   0.534758   0.003779   0.556557   
Gaussian NLLTest             1.880209   2.347087   0.223547    5.31473   
CoverageTest                 0.018918   0.800096   0.009547   0.759166   
IntervalWidthTest            0.054569    3.12606   0.038554   3.130217   
training_time               44.584038  59.945039  59.945039  90.648965   

uncertainty_quantification                                     \
n_samples                          20          50          50   
Statistic                         Std        Mean         Std   
CRPSValidation               0.008753    0.534927    0.002959   
Gaussian NLLValidation       1.528773    1.915489    0.110095   
CoverageValidation           0.016967    0.804567    0.009101   
IntervalWidthValidation        0.0518    3.162884    0.081097   
MSETest                      0.009124    0.885742    0.005015   
EnergyScoreTest              0.006094    0.623946    0.003597   
CRPSTest                     0.008782    0.535566    0.003023   
Gaussian NLLTest             1.516304    1.917412    0.109722   
CoverageTest                  0.01702    0.803627    0.009181   
IntervalWidthTest            0.049552    3.156548    0.080085   
training_time               90.648965  181.454316  181.454316   

uncertainty_quantification scoring-rule-reparam                                \
n_samples                                    3         3         5         5    
Statistic                                  Mean       Std      Mean       Std   
CRPSValidation                          0.47347  0.003678  0.474363  0.003353   
Gaussian NLLValidation                 1.225241  0.008222  1.371462   0.44014   
CoverageValidation                     0.940477  0.002839  0.942409  0.002722   
IntervalWidthValidation                3.258196  0.029861  3.273296  0.027697   
MSETest                                0.863959  0.003844  0.866071  0.003566   
EnergyScoreTest                        0.608104  0.002693  0.609589  0.002497   
CRPSTest                               0.474298  0.003695  0.475138  0.003164   
Gaussian NLLTest                       1.226809  0.008312  1.376768  0.450959   
CoverageTest                           0.940139  0.002973  0.942123  0.002708   
IntervalWidthTest                      3.259391  0.029403

In [18]:
# Initialize an empty DataFrame to store the formatted values
formatted_df = pd.DataFrame()
metrics = results_df.index[-7:] # Extract only test metrics
methods = list(results_df.columns.unique())  # Methods are the top level of the columns MultiIndex

# Create a new DataFrame with the method as the index and metrics as columns
for metric in metrics:
    formatted_df[metric] = [
        f"\\makecell{{{results_df.loc[metric, method].values[0]:.4f} \\\\ ($\\pm$ {results_df.loc[metric, method].values[1]:.4f})}}"
        for method in methods
    ]
formatted_df.index = methods
# Move last column to beginniing
cols = ['training_time'] + list(formatted_df.columns[:-1])
formatted_df = formatted_df[cols]
latex_table = formatted_df.to_latex(escape=False)
#  save to a file
with open(f"{output_path}aggregated_results.tex", "w") as f:
        f.write(latex_table)